# Redundancy Agent – Determining redundancy in several similar signals

In this tutorial we generate four signals and supply the data to a Redundancy Agent.
The agent calculates the best consistent estimate taking into account the
supplied uncertainties. It rejects sensor values that may be erroneous. In this
presented use case the sensors directly measure the measurand.

At the start of the main module we define all important parameters. For
detailed descriptions of the parameters please refer to the
[docstrings of the respective methods](https://agentmet4fof.readthedocs.io/en/latest/agentMET4FOF_agents.html#module-agentMET4FOF.agents.metrological_redundancy_agents).
Then we define the agents and start the network. The network and
the calculated results can be monitored in a browser at the
address http://127.0.0.1:8050/.


In [3]:
# %load redundancy_agent_four_signals.py
import numpy as np

from agentMET4FOF.agents.metrological_base_agents import (
    MetrologicalMonitorAgent,
)
from agentMET4FOF.agents.metrological_redundancy_agents import RedundancyAgent
from agentMET4FOF.agents.metrological_signal_agents import (
    MetrologicalGeneratorAgent,
)
from agentMET4FOF.network import AgentNetwork
from agentMET4FOF.streams.metrological_signal_streams import (
    MetrologicalMultiWaveGenerator,
)


def demonstrate_redundancy_agent_four_signals():
    batch_size = 10
    n_pr = batch_size
    fsam = 100
    intercept = 10
    frequencies = [6, 10, 8, 12]
    phases = [1, 2, 3, 4]
    amplitudes = [0.3, 0.2, 0.5, 0.4]
    exp_unc_abs = 0.2
    probability_limit = 0.95

    # start agent network server
    agent_network: AgentNetwork = AgentNetwork(dashboard_modules=True)

    # Initialize signal generating class outside of agent framework.
    signal_arr = [
        MetrologicalMultiWaveGenerator(
            sfreq=fsam,
            freq_arr=np.array([frequency]),
            ampl_arr=np.array([amplitude]),
            phase_ini_arr=np.array([phase]),
            intercept=intercept,
            value_unc=exp_unc_abs,
        )
        for frequency, amplitude, phase in zip(frequencies, amplitudes, phases)
    ]

    # Data source agents.
    source_agents = []
    sensor_key_list = []
    for count, signal in enumerate(signal_arr):
        sensor_key_list += ["Sensor" + str(count + 1)]
        source_agents += [
            agent_network.add_agent(
                name=sensor_key_list[-1], agentType=MetrologicalGeneratorAgent
            )
        ]
        source_agents[-1].init_parameters(signal=signal, batch_size=batch_size)

    # Redundant data processing agent
    redundancy_name1 = "RedundancyAgent1"
    redundancy_agent1 = agent_network.add_agent(
        name=redundancy_name1, agentType=RedundancyAgent
    )
    redundancy_agent1.init_parameters(
        sensor_key_list=sensor_key_list,
        n_pr=n_pr,
        problim=probability_limit,
        calc_type="lcs",
    )

    # Initialize metrologically enabled plotting agent.
    monitor_agent1 = agent_network.add_agent(
        name="MonitorAgent_SensorValues", agentType=MetrologicalMonitorAgent
    )
    monitor_agent2 = agent_network.add_agent(
        name="MonitorAgent_RedundantEstimate", agentType=MetrologicalMonitorAgent
    )

    # Bind agents.
    for source_agent in source_agents:
        source_agent.bind_output(monitor_agent1)
        source_agent.bind_output(redundancy_agent1)

    redundancy_agent1.bind_output(monitor_agent2)

    # Set all agents states to "Running".
    agent_network.set_running_state()

    # Allow for shutting down the network after execution.
    return agent_network


if __name__ == "__main__":
    demonstrate_redundancy_agent_four_signals()

Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:3333 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:3333
INFO [2021-07-07 14:49:18.993883] (Sensor1): INITIALIZED
INFO [2021-07-07 14:49:19.035303] (Sensor2): INITIALIZED
INFO [2021-07-07 14:49:19.072919] (Sensor3): INITIALIZED
INFO [2021-07-07 14:49:19.110387] (Sensor4): INITIALIZED
INFO [2021-07-07 14:49:19.143509] (RedundancyAgent1): INITIALIZED


/home/ludwig10/code/agentMET4FOF/agentMET4FOF/metrological_streams.py:135: UserWarning:

No uncertainty generator function specified. Setting to default (value_unc = constant, time_unc = 0).




--------------------------------------------------------------
|                                                            |
| Your agent network is starting up. Open your browser and   |
| visit the agentMET4FOF dashboard on http://127.0.0.1:8050/ |
|                                                            |
--------------------------------------------------------------

INFO [2021-07-07 14:49:19.188230] (MonitorAgent_SensorValues): INITIALIZED
INFO [2021-07-07 14:49:19.220150] (MonitorAgent_RedundantEstimate): INITIALIZED
[2021-07-07 14:49:19.245339] (Sensor1): Connected output module: MonitorAgent_SensorValues
[2021-07-07 14:49:19.253119] (Sensor1): Connected output module: RedundancyAgent1
[2021-07-07 14:49:19.261596] (Sensor2): Connected output module: MonitorAgent_SensorValues
[2021-07-07 14:49:19.269916] (Sensor2): Connected output module: RedundancyAgent1
[2021-07-07 14:49:19.278184] (Sensor3): Connected output module: MonitorAgent_SensorValues
[2021-07-07 14:49:19.292819]

[2021-07-07 14:49:21.015850] (MonitorAgent_SensorValues): Received: {'from': 'Sensor1', 'data': array([[0.01      , 0.        , 9.85659121, 0.2       ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7fecad3cea00>, 'senderType': 'MetrologicalGeneratorAgent', 'channel': 'default'}
[2021-07-07 14:49:21.015851] (RedundancyAgent1): Received: {'from': 'Sensor1', 'data': array([[0.01      , 0.        , 9.85659121, 0.2       ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7fecad3d5ee0>, 'senderType': 'MetrologicalGeneratorAgent', 'channel': 'default'}
[2021-07-07 14:49:21.050802] (Sensor2): Pack time: 0.004355
[2021-07-07 14:49:21.078180] (Sensor3): Pack time: 0.000268
[2021-07-07 14:49:21.078841] (Sensor3): Sending: [array([[0.01      , 0.        , 9.33105188, 0.2       ]]), <time_series_metadata.scheme.MetaData object at 0x7fecad4826d0>]
[2021-07-07 14:49:21.062134] (MonitorAgent_SensorValues): Buffer: {'Sensor1': {'data': array([[0.00000000e+00, 0.000000

[2021-07-07 14:49:21.118684] (MonitorAgent_SensorValues): Buffer: {'Sensor1': {'data': array([[0.00000000e+00, 0.00000000e+00, 1.00575868e+01, 2.00000000e-01],
       [1.00000000e-02, 0.00000000e+00, 9.85659121e+00, 2.00000000e-01]]), 'metadata': [<time_series_metadata.scheme.MetaData object at 0x7fecad3ce6a0>, <time_series_metadata.scheme.MetaData object at 0x7fecad3cea00>]}, 'Sensor2': {'data': array([[0.        , 0.        , 9.81226671, 0.2       ],
       [0.01      , 0.        , 9.62458464, 0.2       ]]), 'metadata': [<time_series_metadata.scheme.MetaData object at 0x7fecad3ce7f0>, <time_series_metadata.scheme.MetaData object at 0x7fecad3ceaf0>]}, 'Sensor3': {'data': array([[0.        , 0.        , 9.40049983, 0.2       ],
       [0.01      , 0.        , 9.33105188, 0.2       ]]), 'metadata': [<time_series_metadata.scheme.MetaData object at 0x7fecad3cea30>, <time_series_metadata.scheme.MetaData object at 0x7fecad3ce700>]}, 'Sensor4': {'data': array([[0.        , 0.        , 9.6340

...

[2021-07-07 14:49:27.127213] (MonitorAgent_SensorValues): Tproc: 0.00831
[2021-07-07 14:49:27.122187] (RedundancyAgent1): Tproc: 0.003364
Buffer size is  8 , which is less than  10 .
Not enough data for redundancy agent evaluation.
[2021-07-07 14:49:27.998756] (Sensor1): Pack time: 0.000271
[2021-07-07 14:49:27.999481] (Sensor1): Sending: [array([[0.08     , 0.       , 9.9701676, 0.2      ]]), <time_series_metadata.scheme.MetaData object at 0x7fecad47d550>]
[2021-07-07 14:49:28.000684] (RedundancyAgent1): Received: {'from': 'Sensor1', 'data': array([[0.08     , 0.       , 9.9701676, 0.2      ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7fecad46b1c0>, 'senderType': 'MetrologicalGeneratorAgent', 'channel': 'default'}
[2021-07-07 14:49:28.002045] (MonitorAgent_SensorValues): Received: {'from': 'Sensor1', 'data': array([[0.08     , 0.       , 9.9701676, 0.2      ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7fecad459d00>, 'senderType': 'Metrologic

[2021-07-07 14:49:28.009079] (MonitorAgent_SensorValues): Tproc: 0.006906
[2021-07-07 14:49:28.011280] (RedundancyAgent1): Tproc: 0.010427
[2021-07-07 14:49:28.041395] (Sensor2): Pack time: 0.000193
[2021-07-07 14:49:28.042955] (MonitorAgent_SensorValues): Received: {'from': 'Sensor2', 'data': array([[ 0.08     ,  0.       , 10.3098588,  0.2      ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7fecad3f0130>, 'senderType': 'MetrologicalGeneratorAgent', 'channel': 'default'}
[2021-07-07 14:49:28.041970] (Sensor2): Sending: [array([[ 0.08     ,  0.       , 10.3098588,  0.2      ]]), <time_series_metadata.scheme.MetaData object at 0x7fecad4806d0>]
[2021-07-07 14:49:28.044622] (MonitorAgent_SensorValues): Buffer: {'Sensor1': {'data': array([[0.00000000e+00, 0.00000000e+00, 1.00575868e+01, 2.00000000e-01],
       [1.00000000e-02, 0.00000000e+00, 9.85659121e+00, 2.00000000e-01],
       [2.00000000e-02, 0.00000000e+00, 9.99946200e+00, 2.00000000e-01],
       [3.00000000e-02,

[2021-07-07 14:49:28.044681] (MonitorAgent_SensorValues): Tproc: 0.001629
[2021-07-07 14:49:28.048806] (RedundancyAgent1): Tproc: 0.002818
[2021-07-07 14:49:28.076686] (Sensor3): Pack time: 0.000103
[2021-07-07 14:49:28.076910] (Sensor3): Sending: [array([[ 0.08      ,  0.        , 10.53250935,  0.2       ]]), <time_series_metadata.scheme.MetaData object at 0x7fecad4826d0>]
[2021-07-07 14:49:28.077554] (RedundancyAgent1): Received: {'from': 'Sensor3', 'data': array([[ 0.08      ,  0.        , 10.53250935,  0.2       ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7fecad46b3d0>, 'senderType': 'MetrologicalGeneratorAgent', 'channel': 'default'}
[2021-07-07 14:49:28.080638] (RedundancyAgent1): Buffer: {'Sensor1': array([[0.00000000e+00, 0.00000000e+00, 1.00575868e+01, 2.00000000e-01],
       [1.00000000e-02, 0.00000000e+00, 9.85659121e+00, 2.00000000e-01],
       [2.00000000e-02, 0.00000000e+00, 9.99946200e+00, 2.00000000e-01],
       [3.00000000e-02, 0.00000000e+00, 1.

[2021-07-07 14:49:28.080716] (MonitorAgent_SensorValues): Tproc: 0.00308
[2021-07-07 14:49:28.114989] (Sensor4): Pack time: 0.00015
[2021-07-07 14:49:28.115411] (Sensor4): Sending: [array([[0.08      , 0.        , 9.83409251, 0.2       ]]), <time_series_metadata.scheme.MetaData object at 0x7fecad482700>]
[2021-07-07 14:49:28.116198] (RedundancyAgent1): Received: {'from': 'Sensor4', 'data': array([[0.08      , 0.        , 9.83409251, 0.2       ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7fecad46b3d0>, 'senderType': 'MetrologicalGeneratorAgent', 'channel': 'default'}
[2021-07-07 14:49:28.116198] (MonitorAgent_SensorValues): Received: {'from': 'Sensor4', 'data': array([[0.08      , 0.        , 9.83409251, 0.2       ]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7fecad3f0040>, 'senderType': 'MetrologicalGeneratorAgent', 'channel': 'default'}
[2021-07-07 14:49:28.119805] (MonitorAgent_SensorValues): Buffer: {'Sensor1': {'data': array([[0.00000000e+

[2021-07-07 14:49:28.120010] (MonitorAgent_SensorValues): Tproc: 0.003683
[2021-07-07 14:49:28.119848] (RedundancyAgent1): Tproc: 0.003522
Buffer size is  9 , which is less than  10 .
Not enough data for redundancy agent evaluation.
Buffer size is  9 , which is less than  10 .
Not enough data for redundancy agent evaluation.
